In [1]:
from bs4 import BeautifulSoup as bs
import pandas as pd
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.select import Select
from selenium.common import exceptions 
from time import sleep
from tqdm import tqdm
from transliterate import translit, get_available_language_codes
import re

In [7]:
browser = Chrome()

In [3]:
url_lavka = 'https://lavka.yandex.ru'
url_ozon = 'https://www.ozon.ru/category/supermarket-25000/?__rr=1&miniapp=supermarket'
url_vkus = 'https://vkusvill.ru/?utm_source=admitad&utm_campaign=34f44854d746c3631a876c8ea4f49bec&utm_content=2125038&utm_medium=vakansii'

In [4]:
t = pd.read_excel(r"C:\Users\nikit\Downloads\Тестовое задание.xlsx", sheet_name = ['Москва', 'Ростов-на-Дону'])
Mos = t['Москва'].iloc[2:,0].to_list()
Mos
Moscl = []
for i in Mos:
    Moscl.append(re.sub(r'\s+',' ', i.replace('Самокат ', '')))
    

## Перевод

In [5]:
def transliterate(name):
    if name.split(' ')[-1] == 'г':
        name = name[:-2]
   # Слоаврь с заменами
    slovar = {'а':'a','б':'b','в':'v','г':'g','д':'d','е':'e','ё':'yo',
      'ж':'zh','з':'z','и':'i','й':'j','к':'k','л':'l','м':'m','н':'n',
      'о':'o','п':'p','р':'r','с':'s','т':'t','у':'u','ф':'f','х':'h',
      'ц':'c','ч':'ch','ш':'sh','щ':'sch','ъ':'','ы':'y','ь':'','э':'e',
      'ю':'u','я':'ya', 'А':'A','Б':'B','В':'V','Г':'G','Д':'D','Е':'E','Ё':'YO',
      'Ж':'ZH','З':'Z','И':'I','Й':'I','К':'K','Л':'L','М':'M','Н':'N',
      'О':'O','П':'P','Р':'R','С':'S','Т':'T','У':'U','Ф':'F','Х':'H',
      'Ц':'C','Ч':'CH','Ш':'SH','Щ':'SCH','Ъ':'','Ы':'y','Ь':'','Э':'E',
      'Ю':'U','Я':'YA',',':'','?':'','~':'','!':'','@':'','#':'',
      '$':'','%':'','^':'','&':'','*':'','(':'',')':'','-':'','=':'',
      ':':'',';':'','<':'','>':'','\'':'','"':'','\\':'','/':'','№':'',
      '[':'',']':'','{':'','}':'','ґ':'','ї':'', 'є':'','Ґ':'g','Ї':'i',
      'Є':'e', '—':'', '1':'', '2':'', '3':'', '4':'', '5':'', '6':'', '7':'',
      '8':'', '9':'', '0':'',' ':'_'}
        
   # Циклически заменяем все буквы в строке
    for key in slovar:
        name = name.lower().replace(key, slovar[key])
    if name[-1] == '_':
        name = name[:-1]
    return name

## Создание датафрейма

In [6]:
df = pd.DataFrame(columns = ['Лавка','Озон','Вкус','ЦЛ', 'ЦЛП','ЦО', 'ЦОП','ЦВ', 'ЦВП'], index = Mos)
df

,Лавка,Озон,Вкус,ЦЛ,ЦЛП,ЦО,ЦОП,ЦВ,ЦВП
"Бекон сырокопчёный Дымов нарезка, 150 г",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Из Эквадора: Бананы Самокат 4 шт.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Брауни FitnesShock 15% протеина, без сахара, горячий шоколад, 50 г",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Бумага туалетная Самокат 2 слоя, 8 рулонов",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Бумага туалетная Zewa Natural Comfort белая, 3 слоя, 6 рулонов",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Вода минеральная Borjomi газированная, в стекле, 500 мл",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Вода питьевая Святой Источник негазированная, 5+1 шт. по 1,5 л",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Гуляш Самокат говядина, охлаждённый, 380 г",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Драже M&M’s с арахисом и молочным шоколадом, 145 г",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Картофель белый, мытый, 2 кг",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Общие функции

### Определение лучшего варианта из выдачи

In [8]:
def bestl(t, nm, k, price, full_price, name, weight = 0):
    nmt = transliterate(nm)
    c = 0
    best = ''
    for lt in t:
        link = lt.find_element(By.TAG_NAME, 'a').get_attribute('href')
        price = lt.find_element(By.XPATH, price).text
        full_price = lt.find_element(By.XPATH, full_price).text
        name = lt.find_element(By.XPATH, name).text
        if weight != 0:
            weight = lt.find_element(By.XPATH, weight).text
        if c == 0:
            print(name)
            price_of_best = price 
            full_price_of_best = full_price
            name_of_best = name 
            weight_best = weight
            c += 1
        try:
            nameinlink = link.split(r'/')[-1]
            if nmt.split().issubset(nameinlink.lower()) or nameinlink.split().lower().issubset(nmt):
                print(1)
                price_of_best = price 
                full_price_of_best = full_price
                name_of_best = name 
                weight_best = weight
                break
        except:
            break
    df.iloc[k, 0] = name + ', ' + weight
    df.iloc[k, 3] = price_of_best
    df.iloc[k, 4] = full_price_of_best

SyntaxError: invalid syntax (2636717111.py, line 5)

### Перебор вариантов

In [32]:
def sup(searchbar, check, class_of_supply_el):
    for k in range(len(Moscl)):
        i = Moscl[k]
        searchbar.send_keys(Keys.CONTROL + 'a' + Keys.DELETE)
        searchbar.send_keys(i)
        wait(browser, 15).until(EC.text_to_be_present_in_element((By.XPATH, check), i))
        t = browser.find_elements(By.CLASS_NAME, class_of_supply_el)
        bestl(t, i, k, price, full_price, name, weight)

## Для Лавки

### Установка геолокации

In [31]:
browser.get(url_lavka)
address = wait(browser, 10).until(EC.presence_of_element_located((By.XPATH,"/html/body/div/div[1]/header/div[2]/div[3]/button"))).click()
adrsstr = wait(browser, 10).until(EC.presence_of_element_located((By.XPATH,"/html/body/div[2]/div[3]/div/div/div/div[1]/div[2]/div[1]/div/input")))
adrsstr.send_keys(Keys.CONTROL + 'a' + Keys.DELETE)
adrsstr.send_keys('Москва, Запорожская улица, 5')
sleep(2)
adrsstr.send_keys(Keys.SPACE)
sleep(1)
wait(browser, 15).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div[3]/div/div/div/div[1]/div[2]/div[1]/div/ul/li[1]'))).click()
but = wait(browser, 10).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div[3]/div/div/div/div[1]/div[2]/div[2]/button')))
sleep(2)
but.click()
# for i in Mos:
#     print(i)
#     searchbar.send_keys(i)
#     break

### Данные Лавки

In [35]:
price = '/html/body/div/div[3]/div[1]/main/div/div/div/ul/li[1]/div[1]/div/div/div/div[1]/div[1]/div/div/div/span[4]/span[1]'
full_price = '/html/body/div/div[3]/div[1]/main/div/div/div/ul/li[1]/div[1]/div/div/div/div[1]/div[1]/div/div/div/span[4]/span[2]/span[1]'
name = '/html/body/div/div[3]/div[1]/main/div/div/div/ul/li[1]/div[1]/div/div/div/div[1]/div[1]/div/h3'
weight = '/html/body/div/div[3]/div[1]/main/div/div/div/ul/li[1]/div[1]/div/div/div/div[1]/div[1]/span/span/span/span/span'
searchbar = wait(browser, 10).until(EC.presence_of_element_located((By.ID,"search-input-id")))
check = '/html/body/div/div[3]/div[1]/main/div/div/div/div[1]/div/div/h2'
class_of_supply_el = 'c8uba0o'
sup(searchbar, check, class_of_supply_el)

Бекон Дымов
Спелый банан
Бисквит проте­иновый FitnesShock брауни
Туалет­ная бумага Zewa Natural Comfort 3 слоя
Туалет­ная бумага Zewa Natural Comfort 3 слоя
Вода минераль­ная газиро­ванная Borjomi
Вода негази­рован­ная Святой источ­ник
Гуляш из говяди­ны Из Лавки
Драже M&M's арахис-шоколад
Карто­фель белый ранний
Квас хлебный Лидский
Чизкейк малина Из Лавки
Колба­са Доктор­ская Клинский
Perfect Fit курица в соусе для стери­лизован­ных кошек
Котле­ты куриные с пюре Из Лавки
Круас­сан c малиной и завар­ным кремом
Манго жёлтое
Молоко безлак­тозное 1,5% Просто­кваши­но
Мороже­ное Дыня Bahroma
Добрый Cola
Шорли яблоч­ный Из Лавки
Огурцы луховиц­кие
Салат оливье с курицей Из Лавки
Салат оливье с колба­сой Из Лавки


In [36]:
df

,Лавка,Озон,Вкус,ЦЛ,ЦЛП,ЦО,ЦОП,ЦВ,ЦВП
"Бекон сырокопчёный Дымов нарезка, 150 г","Бекон Дымов, 150 г",NaN,NaN,169 ₽,239 ₽,NaN,NaN,NaN,NaN
Из Эквадора: Бананы Самокат 4 шт.,"Спелый банан, 1 шт.",NaN,NaN,23 ₽,39 ₽,NaN,NaN,NaN,NaN
"Брауни FitnesShock 15% протеина, без сахара, горячий шоколад, 50 г","Бисквит проте­иновый FitnesShock брауни, 50 г",NaN,NaN,83 ₽,139 ₽,NaN,NaN,NaN,NaN
"Бумага туалетная Самокат 2 слоя, 8 рулонов","Туалет­ная бумага Zewa Natural Comfort 3 слоя,...",NaN,NaN,189 ₽,259 ₽,NaN,NaN,NaN,NaN
"Бумага туалетная Zewa Natural Comfort белая, 3 слоя, 6 рулонов","Туалет­ная бумага Zewa Natural Comfort 3 слоя,...",NaN,NaN,189 ₽,259 ₽,NaN,NaN,NaN,NaN
"Вода минеральная Borjomi газированная, в стекле, 500 мл","Вода минераль­ная газиро­ванная Borjomi, 500 мл",NaN,NaN,89 ₽,149 ₽,NaN,NaN,NaN,NaN
"Вода питьевая Святой Источник негазированная, 5+1 шт. по 1,5 л","Вода негази­рован­ная Святой источ­ник, 5 л",NaN,NaN,135 ₽,149 ₽,NaN,NaN,NaN,NaN
"Гуляш Самокат говядина, охлаждённый, 380 г","Гуляш из говяди­ны Из Лавки, 400 г",NaN,NaN,239 ₽,399 ₽,NaN,NaN,NaN,NaN
"Драже M&M’s с арахисом и молочным шоколадом, 145 г","Драже M&M's арахис-шоколад, 360 г",NaN,NaN,379 ₽,469 ₽,NaN,NaN,NaN,NaN
"Картофель белый, мытый, 2 кг","Карто­фель белый ранний, 1 кг",NaN,NaN,59 ₽,99 ₽,NaN,NaN,NaN,NaN


## Для озон

### Установка геолокации озон

In [48]:
browser.get(url_ozon)
browser.refresh()
sleep(1)
x_but = wait(browser, 10).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div/div[1]/div[1]/div/div/div/div/div/div/div/div')))
sleep(0.1)
x_but.click()
try:
    adrstro = wait(browser, 10).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[4]/div/div[2]/div/div/div/div/div/div/div[1]/div/div/div/div[1]/div/div/form/div/div/fieldset/div[1]/div/div/div/label/div[1]/div/textarea')))
    wait(browser, 15).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[4]/div/div[2]/div/div/div/div/div/div/div[1]/div/div/div/div[1]/div/div/form/div/div/fieldset/div[1]/div/div/div/label/div[1]/div/div/div/button/span'))).click()
    adrstro.send_keys('Москва, Запорожская улица, 5')
    sleep(0.5)
    adrstro.send_keys(Keys.ENTER)
    sleep(1)
    wait(browser, 15).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[5]/div/div/div/div'))).click()
    sleep(2)
    wait(browser, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[4]/div/div[2]/div/div/div/div/div/div/div[1]/div/div/div/div[2]/button'))).click()
    save_but = wait(browser, 10).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[4]/div/div[2]/div/div/div/div/div/div/div[1]/div/div/div/div[2]/button')))
    sleep(0.1)
    save_but.click()
except:
     wait(browser, 15).until(EC.presence_of_element_located((By.XPATH,'/html/body/div[4]/div/div[2]/div/div/button/div'))).click()

### Сбор информации о товарах